In [1]:
from collections import deque, defaultdict
from UserClasses import generateTree, generateList, ListNode, TreeNode
from math import sin, cos, pi
from functools import reduce
import re
import heapq

In [34]:
#590. N-ary Tree Postorder Traversal
class Postorder(object):        
    def postorder(self, root):
        result = []
        dfs_stack = deque([root])
        while dfs_stack:
            node = dfs_stack.pop()
            result.append(node.val)
            if node.children:
                dfs_stack.extend(node.children)
        result.reverse()
        return result    

In [50]:
#1514. Path with Maximum Probability
class MaxProbability(object):
    def _getGraph(self, uEdges, uSuccProb):
        graph = defaultdict(list)
        for index, edge in enumerate(uEdges):
            graph[edge[0]].append((uSuccProb[index], edge[1]))
            graph[edge[1]].append((uSuccProb[index], edge[0]))
        return graph
        
    def maxProbability(self, n, edges, succProb, start_node, end_node):
        graph = self._getGraph(edges, succProb)
        node_prob = {index: -1 if index == start_node else 0 for index in range(n)}
        max_node = []
        heapq.heapify(max_node)
        heapq.heappush(max_node, (node_prob[start_node], start_node))
        while max_node:
            cur_prob, cur_node = heapq.heappop(max_node)
            if cur_node in graph and cur_node != end_node:
                for next_prob, next_node in graph[cur_node]:
                    if next_prob * cur_prob < node_prob[next_node]:
                        node_prob[next_node] = next_prob * cur_prob
                        heapq.heappush(max_node, (node_prob[next_node], next_node))
        return node_prob[end_node] * -1

In [53]:
#145. Binary Tree Postorder Traversal
class PostorderTraversal(object):
    def postorderTraversal(self, root):
        dfs_stack = deque([])
        if root:
            dfs_stack.append(root)
        result = []
        right_node = False
        while dfs_stack:
            cur_node = dfs_stack.pop() if not right_node else dfs_stack.popleft()
            result.append(cur_node.val)
            right_node = False
            if cur_node.right:
                dfs_stack.appendleft(cur_node.right)
                right_node = True
            if cur_node.left:
                dfs_stack.append(cur_node.left)
        result.reverse()
        return result

In [22]:
#1905. Count Sub Islands
class СountSubIslands(object):
    def _goGrid(self, grid, main_grid, start_row_index, start_col_index):
        sub_island_flag = True
        dfs_stack = deque([[start_row_index, start_col_index]])
        while dfs_stack:
            coord = dfs_stack.pop()
            row_index, col_index = coord[0], coord[1]
            grid[row_index][col_index] = 0
            if not main_grid[row_index][col_index]:
                sub_island_flag = False
            if (row_index > 0) and (grid[row_index-1][col_index]):
                dfs_stack.append([row_index-1, col_index])
            if (row_index < len(grid) - 1) and (grid[row_index+1][col_index]):
                dfs_stack.append([row_index+1, col_index])
            if (col_index > 0) and (grid[row_index][col_index-1]):
                dfs_stack.append([row_index, col_index-1])
            if (col_index < len(grid[0]) - 1) and (grid[row_index][col_index+1]):
                dfs_stack.append([row_index, col_index+1])
        return sub_island_flag
            
    def countSubIslands(self, grid1, grid2):
        result = 0
        for row_index in range(len(grid2)):
            for col_index in range(len(grid2[0])):
                if grid2[row_index][col_index]:
                    result += self._goGrid(grid2, grid1, row_index, col_index)
        return result

In [62]:
#947. Most Stones Removed with Same Row or Column
class RemoveStones(object):
    def _getGraph(self, uStones):
        graph = {}
        for stone in uStones:
            graph[tuple(stone)] = set()
        for index, main_stone in enumerate(uStones[:-1]):
            for add_stone in uStones[index+1:]:
                if (main_stone[0] == add_stone[0]) or (main_stone[1] == add_stone[1]):
                    stone_1, stone_2 = tuple(main_stone), tuple(add_stone)
                    graph[stone_1].add(stone_2)
                    graph[stone_2].add(stone_1)
        return graph

    def _dfs(self, uGraph):
        dfs_stack = deque([list(uGraph.keys())[0]])
        while dfs_stack:
            stone = dfs_stack.pop()
            if stone in uGraph:
                dfs_stack.extend(uGraph[stone])
                uGraph.pop(stone)
                    
    def removeStones(self, stones):
        graph = self._getGraph(stones)
        remain_stones = 0
        while graph:
            self._dfs(graph)
            remain_stones += 1
        return len(stones) - remain_stones

In [18]:
#1894. Find the Student that Will Replace the Chalk
class ChalkReplacer(object):
    def _getPrefixSum(self, uChalk):
        prefix_sum = [uChalk[0]]
        for num in uChalk[1:]:
            prefix_sum.append(prefix_sum[-1] + num)
        return prefix_sum

    def _binarySearch(self, uPrefixSum, uK):
        if uK < uPrefixSum[0]:
            return 0
        left_pointer, right_pointer = 0, len(uPrefixSum) -  1
        while right_pointer - left_pointer > 1:
            middle_pointer = (right_pointer + left_pointer) // 2
            if uPrefixSum[middle_pointer] <= uK:
                left_pointer = middle_pointer
            else:
                right_pointer = middle_pointer
        return right_pointer
        
    def chalkReplacer(self, chalk, k):
        prefix_sum = self._getPrefixSum(chalk)
        k -= prefix_sum[-1] * (k // prefix_sum[-1])
        return self._binarySearch(prefix_sum, k)

In [93]:
#2699. Modify Graph Edge Weights
class ModifiedGraphEdges(object):
    def _getGraph(self, pN, pEdges):
        graph = {index: dict() for index in range(pN)}
        for edge in pEdges:
            graph[edge[0]][edge[1]] = edge[2]
            graph[edge[1]][edge[0]] = edge[2]
        return graph

    def _getResGraph(self, pGraph, pSource, pDestination, pTarget):
        modified_edges = deque([])
        res_graph = {index: dict() for index in pGraph}
        for cur_node in pGraph:
            for next_node in pGraph[cur_node]:
                if pGraph[cur_node][next_node] == -1:
                    if cur_node < next_node:
                        modified_edges.append([cur_node, next_node])
                    res_graph[cur_node][next_node] = 1
                else:
                    res_graph[cur_node][next_node] = pGraph[cur_node][next_node]
        while True:
            min_path = self._algoDijkstra(res_graph, pSource, pDestination)
            if min_path == pTarget:
                return res_graph
            elif modified_edges:
                mod_edge = modified_edges.popleft()
                res_graph[mod_edge[0]][mod_edge[1]] += pTarget - min_path
                res_graph[mod_edge[1]][mod_edge[0]] += pTarget - min_path
            else: 
                return {}
        
    def _algoDijkstra(self, pGraph, pSource, pDestination):
        min_node_vals = []
        node_vals = {index: None for index in pGraph if index != pSource}
        checked_nodes = set()
        heapq.heapify(min_node_vals)
        heapq.heappush(min_node_vals, (0, pSource))
        while min_node_vals:
            cur_node_val, cur_node = heapq.heappop(min_node_vals)
            if cur_node != pDestination:
                checked_nodes.add(cur_node)
                for next_node in pGraph[cur_node]:
                    if (next_node not in checked_nodes
                        and pGraph[cur_node][next_node] != -1
                        and (node_vals[next_node] is None 
                             or pGraph[cur_node][next_node] + cur_node_val < node_vals[next_node])):
                        node_vals[next_node] = pGraph[cur_node][next_node] + cur_node_val
                        heapq.heappush(min_node_vals, (node_vals[next_node], next_node))
        return node_vals[pDestination]  

    def _getEdges(self, graph):
        result = []
        for cur_node in graph:
            for next_node in graph[cur_node]:
                if cur_node < next_node:
                    result.append([cur_node, next_node, graph[cur_node][next_node]])
        return result
    
    def modifiedGraphEdges(self, n, edges, source, destination, target):
        graph = self._getGraph(n, edges)
        min_path = self._algoDijkstra(graph, source, destination)
        if min_path and min_path < target:
            return []
        else:
            res_graph = self._getResGraph(graph, source, destination, target)
            return self._getEdges(res_graph)

In [41]:
#1945. Sum of Digits of String After Convert
class GetLucky(object):
    def getLucky(self, s, k):
        num_str = []
        for char in s:
            num_str.extend(str(ord(char) - 96))
        index = 0
        while True:
            num = reduce(lambda x, y: x + int(y), num_str, 0)
            index += 1
            if index == k:
                return num
            num_str = []
            num_str.extend(str(num))

In [23]:
#2028. Find Missing Observations
class MissingRolls(object):
    def missingRolls(self, rolls, mean, n):
        unknown_rolls_sum = mean * (n + len(rolls)) - sum(rolls)
        if 1.0 <= (unknown_rolls_sum / n) <= 6.0:
            result = [unknown_rolls_sum // n for _ in range(n)]
            ostat = unknown_rolls_sum - result[0] * n
            index = 0
            while ostat:
                if ostat > 6 - result[index]:
                    ostat -= 6 - result[index]
                    result[index] = 6
                else:
                    result[index] += ostat
                    ostat = 0
                index += 1
            return result
        else:
            return []

In [36]:
#874. Walking Robot Simulation
class RobotSim(object):
    def _getObstaclesDict(self, pObstacles):
        obstacles_dict = {}
        for obstacle in pObstacles:
            if obstacle[0] not in obstacles_dict:
                obstacles_dict[obstacle[0]] = set()
            obstacles_dict[obstacle[0]].add(obstacle[1])
        return obstacles_dict
    
    def _turnAngle(self, pAngle, pCommande):
        if pCommande == -2:
            pAngle += 90
            return pAngle % 360
        else:
            pAngle -= 90
            return pAngle if pAngle >= 0 else 270

    def _changePosition(self, pPosition, pAngle, pSteps, pObstacles):
        result = pPosition
        for _ in range(pSteps):
            new_position = list(result)
            if pAngle in (0, 180):
                new_position[0] += -1 if pAngle == 180 else 1
            else:
                new_position[1] += 1 if pAngle == 90 else -1
            if new_position[0] in pObstacles and new_position[1] in pObstacles[new_position[0]]:
                return result
            result = new_position
        return result
        
    def robotSim(self, commands, obstacles):
        obstacles_dict = self._getObstaclesDict(obstacles)
        cur_angle = 90
        cur_position = [0, 0]
        result = 0
        for command in commands:
            if command < 0:
                cur_angle = self._turnAngle(cur_angle, command)
            else:
                cur_position = self._changePosition(cur_position, cur_angle, command, obstacles_dict)
                result = max(result, cur_position[0] ** 2 + cur_position[1] ** 2 )
        return result         

In [29]:
#3217. Delete Nodes From Linked List Present in Array
class ModifiedList(object):
    def modifiedList(self, nums, head):
        exclude_nums = set(nums)
        new_head, cur_node = None, None
        while head:
            if head.val not in exclude_nums:
                if new_head is None:
                    new_head = head
                    cur_node = new_head
                else:
                    cur_node.next = head
                    cur_node = cur_node.next
            head = head.next
        if cur_node.next and  cur_node.next.val in exclude_nums:
            cur_node.next = None
        return new_head

In [24]:
#2326. Spiral Matrix IV
class SpiralMatrix(object):
    def _nextCell(self, pCurRow, pCurCol, pTurn, pM, pN):
        if pCurRow == pTurn:
            pCurCol += 1
            if pCurCol > pN - 1 - pTurn:
                pCurCol -= 1
                pCurRow += 1
        elif pCurRow == pM - 1 - pTurn:
            pCurCol -= 1
        
    def spiralMatrix(self, m, n, head):
        res_matrix = [[-1 for _ in range(n)] for _ in range(m)]
        cur_turn = 0
        while cur_turn <= (m - 1) // 2:
            cur_row = cur_turn
            for cur_col in range(cur_turn, n-cur_turn):
                if not head:
                    break
                res_matrix[cur_row][cur_col] = head.val
                head = head.next
            cur_col = n - 1 - cur_turn
            for cur_row in range(cur_turn+1, m-cur_turn):
                if not head:
                    break
                res_matrix[cur_row][cur_col] = head.val
                head = head.next
            cur_row = m - 1 - cur_turn
            for cur_col in range(n-2-cur_turn, cur_turn-1, -1):
                if not head:
                    break
                res_matrix[cur_row][cur_col] = head.val
                head = head.next
            cur_col = cur_turn
            for cur_row in range(m-2-cur_turn, cur_turn, -1):
                if not head:
                    break
                res_matrix[cur_row][cur_col] = head.val
                head = head.next
            cur_turn += 1
        return res_matrix

In [24]:
#2807. Insert Greatest Common Divisors in Linked List
class InsertGreatestCommonDivisors(object):
    def _findGCD(self, pNum1, pNum2):
        greater, less = max(pNum1, pNum2), min(pNum1, pNum2)
        while less:
            temp_greater = less
            less = greater - (less * (greater // less))
            greater = temp_greater
        return greater
    
    def insertGreatestCommonDivisors(self, head):
        res = head
        while head:
            if head.next:
                head.next = ListNode(self._findGCD(head.val, head.next.val), head.next)
                head = head.next
            head = head.next
        return res

In [48]:
#1367. Linked List in Binary Tree
class IsSubPath(object):
    def _getListVals(self, pHead):
        result = ''
        while pHead:
            result += str(pHead.val)
            pHead = pHead.next
        return result
    
    def _getRoutes(self, pRoot):
        result = []
        dfs_stack = deque([(str(pRoot.val), pRoot)])
        while dfs_stack:
            cur_route, cur_node = dfs_stack.pop()
            if not cur_node.right and not cur_node.left:
                result.append(cur_route)
                continue
            if cur_node.right:
                new_route = cur_route + str(cur_node.right.val)
                dfs_stack.append((new_route, cur_node.right))
            if cur_node.left:
                new_route = cur_route + str(cur_node.left.val)
                dfs_stack.append((new_route, cur_node.left))
        return result
        
    def isSubPath(self, head, root):
        target_path = self._getListVals(head)
        routes = self._getRoutes(root)
        for route in routes:
            if route.find(target_path) >= 0:
                return True
        return False       

In [18]:
#725. Split Linked List in Parts
class SplitListToParts(object):
    def _getListSize(self, pHead):
        result = 0
        while pHead:
            result += 1
            pHead = pHead.next
        return result

    def _getPartsSize(self, pHeadSize, pK):
        result = [pHeadSize // pK for _ in range(pK)]
        remainder = pHeadSize % pK
        for index in range(len(result)):
            if remainder:
                result[index] += 1
                remainder -= 1
            else: 
                break
        return result

    def _getParts(self, pPartsSize, pHead):
        result = []
        cur_part = 0
        new_part_flag = True
        while pHead:
            if new_part_flag:
                result.append(pHead)
                new_part_flag = False
            pPartsSize[cur_part] -= 1
            if not pPartsSize[cur_part]:
                cur_part += 1
                new_part_flag = True
                temp_node = pHead.next
                pHead.next = None
                pHead = temp_node
            else:
                pHead = pHead.next
        for _ in range(len(pPartsSize) - cur_part):
            result.append(None)
        return result
        
    def splitListToParts(self, head, k):
        head_size = self._getListSize(head)
        parts_size = self._getPartsSize(head_size, k)
        return self._getParts(parts_size, head)

In [27]:
#2220. Minimum Bit Flips to Convert Number
class MinBitFlips(object):
    def minBitFlips(self, start, goal):
        start_bin, goal_bin = bin(start)[2:], bin(goal)[2:]
        if len(start_bin) > len(goal_bin):
            goal_bin = '0' * (len(start_bin) - len(goal_bin)) + goal_bin
        elif len(start_bin) < len(goal_bin):
            start_bin = '0' * (len(goal_bin) - len(start_bin)) + start_bin
        result = 0
        for index in range(len(start_bin)):
            if start_bin[index] != goal_bin[index]:
                result += 1
        return result      

In [13]:
#1684. Count the Number of Consistent Strings
class CountConsistentStrings(object):
    def countConsistentStrings(self, allowed, words):
        allowed_set = set(allowed)
        result = 0
        for word in words:
            word_set = set(word)
            if not word_set - allowed_set:
                result += 1
        return result                  

In [11]:
#1310. XOR Queries of a Subarray
class XorQueries(object):
    def _getPrefixXOR(self, pArr):
        result = [0]
        for num in pArr:
            result.append(result[-1] ^ num)
        return result
        
    def xorQueries(self, arr, queries):
        prefixXOR = self._getPrefixXOR(arr)
        result = []
        for query in queries:
            result.append(prefixXOR[query[1]+1] ^ prefixXOR[query[0]])
        return result

In [54]:
#539. Minimum Time Difference
class FindMinDifference(object):
    def _convertTime(self, pTime):
        time_arr = [int(time) for time in pTime.split(':')]
        return time_arr[0] * 60 + time_arr[1]
        
    def findMinDifference(self, timePoints):
        minutes_arr = [self._convertTime(time) for time in timePoints]
        minutes_arr.sort()
        result = 1440 - minutes_arr[-1] + minutes_arr[0]
        for index in range(1, len(minutes_arr)):
            result = min(result, minutes_arr[index] - minutes_arr[index-1])
        return result

In [4]:
#884. Uncommon Words from Two Sentences
class UncommonFromSentences(object):
    def uncommonFromSentences(self, s1, s2):
        string_set_1 = set()
        string_set_2 = set()
        for string in s1.split():
            if string in string_set_1:
                string_set_2.add(string)
            else:
                string_set_1.add(string)
        for string in s2.split():
            if string in string_set_2:
                string_set_1.add(string)
            else:
                string_set_2.add(string)
        return list(string_set_1 ^ string_set_2)

In [78]:
#179. Largest Number
class LargestNumber(object):
    def _getGreater(self, pLeft, pRight):
        left_right = str(pLeft[0]) + str(pRight[0])
        right_left = str(pRight[0]) + str(pLeft[0])
        for index in range(len(left_right)):
            if left_right[index] > right_left[index]:
                return pLeft.popleft()
            elif right_left[index] > left_right[index]:
                return pRight.popleft()
        return pLeft.popleft()
    
    def _merge(self, pArray, pLeftIndex, pRightIndex):
        middle_index = pLeftIndex + (pRightIndex - pLeftIndex) // 2
        left = deque(pArray[pLeftIndex:middle_index+1])
        right = deque(pArray[middle_index+1:pRightIndex+1])
        result = []
        while left and right:
            result.append(self._getGreater(left, right))
        if left:
            result.extend(left)
        else:
            result.extend(right)
        pArray[pLeftIndex:pRightIndex+1] = result
        
    def _mergeSort(self, pArray, pLeftIndex, pRightIndex):
        if pLeftIndex != pRightIndex:
            middle_index = pLeftIndex + (pRightIndex - pLeftIndex) // 2
            self._mergeSort(pArray, pLeftIndex, middle_index)
            self._mergeSort(pArray, middle_index + 1, pRightIndex)
        self._merge(pArray, pLeftIndex, pRightIndex)
        
    def largestNumber(self, nums):
        self._mergeSort(nums, 0, len(nums) - 1)
        result = ''.join(map(lambda x: str(x), nums))
        while len(result) > 1:
            if result[0] == '0':
                result = result[1:]
            else:
                break
        return result

In [ ]:
#214. Shortest Palindrome
class ShortestPalindrome(object):
    def shortestPalindrome(self, s):
        end_index = len(s)
        while s[:end_index] != s[end_index-1::-1]:
            end_index -= 1
        return s[:end_index-1:-1] + s

In [ ]:
#2419. Longest Subarray With Maximum Bitwise AND
class LongestSubarray(object):
    def longestSubarray(self, nums):
        max_num = max(nums)
        result = 0
        cur_length = 0
        for num in nums:
            if num == max_num:
                cur_length += 1
            else:
                result = max(result, cur_length)
                cur_length = 0
        result = max(result, cur_length)
        return result

In [ ]:
#Сортировка слиянием
class MergeSort:
    def __init__(self, pArray):
        self._array = pArray

    def __repr__(self):
        return str(self._array)
        
    def _getGreater(self, pLeft, pRight):
        if pLeft[0] >= pRight[0]:
            return pLeft.popleft()
        else:
            return pRight.popleft()
        
    def _merge(self, pArray, pLeftIndex, pRightIndex):
        middle_index = pLeftIndex + (pRightIndex - pLeftIndex) // 2
        left = deque(pArray[pLeftIndex:middle_index+1])
        right = deque(pArray[middle_index+1:pRightIndex+1])
        result = []
        while left and right:
            result.append(self._getGreater(left, right))
        if left:
            result.extend(left)
        else:
            result.extend(right)
        pArray[pLeftIndex:pRightIndex+1] = result
        
    def _mergeSort(self, pArray, pLeftIndex, pRightIndex):
        if pLeftIndex != pRightIndex:
            middle_index = pLeftIndex + (pRightIndex - pLeftIndex) // 2
            self._mergeSort(pArray, pLeftIndex, middle_index)
            self._mergeSort(pArray, middle_index + 1, pRightIndex)
        self._merge(pArray, pLeftIndex, pRightIndex)

    def makeSort(self):
        self._mergeSort(self._array, 0, len(self._array) - 1)